In [1]:
from transformers import Qwen3VLForConditionalGeneration, AutoProcessor

# default: Load the model on the available device(s)
model = Qwen3VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen3-VL-2B-Instruct", dtype="auto", device_map="auto"
)

/home/bongo/anaconda3/envs/qwen3/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:


# We recommend enabling flash_attention_2 for better acceleration and memory saving, especially in multi-image and video scenarios.
# model = Qwen3VLForConditionalGeneration.from_pretrained(
#     "Qwen/Qwen3-VL-2B-Instruct",
#     dtype=torch.bfloat16,
#     attn_implementation="flash_attention_2",
#     device_map="auto",
# )

processor = AutoProcessor.from_pretrained("Qwen/Qwen3-VL-2B-Instruct")

action_text = 'ride'
target_item = 'bicycle'
# 1. 질문 구성: 명확하게 Yes/No 대답을 유도
# Qwen 프롬프트 템플릿에 맞춰 질문
query = f"Is the person {action_text} {target_item} in this image? Answer with Yes or No."
image_path = "/home/bongo/porter_notebook/research/WSAG-PLSP/AGD20K/Seen/trainset/exocentric/push/bicycle/push_bicycle_007122.jpg"

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": image_path,
            },
            {"type": "text", "text": query},
        ],
    }
]

# Preparation for inference
inputs = processor.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_dict=True,
    return_tensors="pt"
)
inputs = inputs.to(model.device)

# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=128)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)


['Yes']


In [4]:
image_path = "/home/bongo/porter_notebook/research/WSAG-PLSP/AGD20K/Seen/trainset/exocentric/push/bicycle/push_bicycle_007122.jpg"

# image_path = "/home/bongo/porter_notebook/research/WSAG-PLSP/AGD20K/Seen/trainset/exocentric/push/bicycle/push_bicycle_012603.jpg"

In [10]:
import os
import torch
import pandas as pd
import numpy as np

In [6]:
"""
"Is the person [action]?" 질문에 대해 
모델이 'Yes'라고 대답할 확률을 계산합니다.
"""
action_text = 'eat'
target_item = 'bicycle'


image_path = "/home/bongo/porter_notebook/research/WSAG-PLSP/AGD20K/Seen/trainset/exocentric/push/bicycle/push_bicycle_007122.jpg"

# 2. 질문 구성
# "Answer with Yes or No."를 명시하여 모델이 다른 말을 할 확률을 줄입니다.
query = f"Is the person {action_text} {target_item} in this image? Answer with Yes or No."

messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image_path},
            {"type": "text", "text": query},
        ],
    }
]

# 3. 입력 처리 (Generation 코드와 동일한 방식 사용)
# tokenize=True로 설정하여 바로 텐서로 변환합니다.
inputs = processor.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True, # <|im_start|>assistant\n 까지 붙여줌
    return_dict=True,
    return_tensors="pt"
)
inputs = inputs.to(model.device)

# 4. Forward Pass (추론)
# model.generate() 대신 model()을 호출하여 확률 분포(Logits)를 얻습니다.
with torch.no_grad():
    outputs = model(**inputs)
    
    # 5. 마지막 토큰의 Logits 추출
    # outputs.logits 형태: [Batch_Size, Sequence_Length, Vocab_Size]
    # 우리는 문장의 맨 마지막 토큰(다음에 올 단어 예측)의 확률이 필요합니다.
    next_token_logits = outputs.logits[:, -1, :]
    
    # 6. 'Yes'와 'No' 토큰 ID 찾기
    # Qwen 토크나이저에서 "Yes"와 "No"의 ID를 찾습니다.
    # add_special_tokens=False로 해야 순수 단어 ID만 나옵니다.
    yes_token_id = processor.tokenizer.encode("Yes", add_special_tokens=False)[0]
    no_token_id = processor.tokenizer.encode("No", add_special_tokens=False)[0]
    
    # 7. 점수 추출
    yes_score = next_token_logits[0, yes_token_id].item()
    no_score = next_token_logits[0, no_token_id].item()
    
    # 8. 확률 계산 (Softmax)
    # 전체 단어장이 아니라, 오직 Yes와 No 두 후보군 사이에서의 확률을 구합니다.
    probs = torch.softmax(torch.tensor([yes_score, no_score]), dim=0)
    yes_prob = probs[0].item()
    no_prob = probs[1].item()

# 결과 출력
print(f"Query: {query}")
print(f"Action Check: {action_text}")
print(f"probability(Yes): {yes_prob:.4f}")
print(f"probability(No): {no_prob:.4f}")

if yes_prob > 0.5:
    print(f"✅ 판단: 맞습니다 (True) : {yes_prob}")
else:
    print(f"❌ 판단: 아닙니다 (False) : {yes_prob}")

Query: Is the person eat bicycle in this image? Answer with Yes or No.
Action Check: eat
probability(Yes): 0.8355
probability(No): 0.1645
✅ 판단: 맞습니다 (True) : 0.8354835510253906


In [8]:
def get_yes_no_score(image_path, action_text,target_item):
    query = f"Is the person {action_text} {target_item} in this image? Answer with Yes or No."

    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image", "image": image_path},
                {"type": "text", "text": query},
            ],
        }
    ]

    # 3. 입력 처리 (Generation 코드와 동일한 방식 사용)
    # tokenize=True로 설정하여 바로 텐서로 변환합니다.
    inputs = processor.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True, # <|im_start|>assistant\n 까지 붙여줌
        return_dict=True,
        return_tensors="pt"
    )
    inputs = inputs.to(model.device)

    # 4. Forward Pass (추론)
    # model.generate() 대신 model()을 호출하여 확률 분포(Logits)를 얻습니다.
    with torch.no_grad():
        outputs = model(**inputs)
        
        # 5. 마지막 토큰의 Logits 추출
        # outputs.logits 형태: [Batch_Size, Sequence_Length, Vocab_Size]
        # 우리는 문장의 맨 마지막 토큰(다음에 올 단어 예측)의 확률이 필요합니다.
        next_token_logits = outputs.logits[:, -1, :]
        
        # 6. 'Yes'와 'No' 토큰 ID 찾기
        # Qwen 토크나이저에서 "Yes"와 "No"의 ID를 찾습니다.
        # add_special_tokens=False로 해야 순수 단어 ID만 나옵니다.
        yes_token_id = processor.tokenizer.encode("Yes", add_special_tokens=False)[0]
        no_token_id = processor.tokenizer.encode("No", add_special_tokens=False)[0]
        
        # 7. 점수 추출
        yes_score = next_token_logits[0, yes_token_id].item()
        no_score = next_token_logits[0, no_token_id].item()
        
        # 8. 확률 계산 (Softmax)
        # 전체 단어장이 아니라, 오직 Yes와 No 두 후보군 사이에서의 확률을 구합니다.
        probs = torch.softmax(torch.tensor([yes_score, no_score]), dim=0)
        yes_prob = probs[0].item()
        no_prob = probs[1].item()

    # 결과 출력
    print(f"Query: {query}")
    print(f"Action Check: {action_text}")
    print(f"probability(Yes): {yes_prob:.4f}")
    print(f"probability(No): {no_prob:.4f}")

    if yes_prob > 0.5:
        print(f"✅ {action_text} 판단: 맞습니다 (True)")
    else:
        print("❌ {action_text}  판단: 아닙니다 (False)")

    return yes_prob

In [9]:
AGD20K_VERBS = [
    "beat", "brush_with", "catch", "cut_with", "drink_with", "hit", "jump", "lie_on", "look_out", "pack",
    "pick_up", "push", "sip", "stick", "swing", "talk_on", "throw", "wash",
    "boxing", "carry", "cut", "drag", "eat", "hold", "kick", "lift", "open", "peel", "pour", "ride",
    "sit_on", "stir", "take_photo", "text_on", "type_on", "write"
]
results = {}

for action in AGD20K_VERBS:
    # action 텍스트를 자연스럽게 변형 (필요시)
    # 예: "jumping", "holding a cup" 등 현재진행형 권장
    score = get_yes_no_score(image_path, action, target_item = 'bicyle')
    results[action] = score
    # print(f"{action}: {score:.4f}")

# Top-3 결과 출력
sorted_results = sorted(results.items(), key=lambda x: x[1], reverse=True)
print("\n🏆 Top 3 Predictions:")
for act, prob in sorted_results[:3]:
    print(f"Checking {act}... Probability: {prob*100:.2f}%")

Query: Is the person beat bicyle in this image? Answer with Yes or No.
Action Check: beat
probability(Yes): 0.9399
probability(No): 0.0601
✅ beat 판단: 맞습니다 (True)
Query: Is the person brush_with bicyle in this image? Answer with Yes or No.
Action Check: brush_with
probability(Yes): 0.7982
probability(No): 0.2018
✅ brush_with 판단: 맞습니다 (True)
Query: Is the person catch bicyle in this image? Answer with Yes or No.
Action Check: catch
probability(Yes): 0.8355
probability(No): 0.1645
✅ catch 판단: 맞습니다 (True)
Query: Is the person cut_with bicyle in this image? Answer with Yes or No.
Action Check: cut_with
probability(Yes): 0.9325
probability(No): 0.0675
✅ cut_with 판단: 맞습니다 (True)
Query: Is the person drink_with bicyle in this image? Answer with Yes or No.
Action Check: drink_with
probability(Yes): 0.4378
probability(No): 0.5622
❌ {action_text}  판단: 아닙니다 (False)
Query: Is the person hit bicyle in this image? Answer with Yes or No.
Action Check: hit
probability(Yes): 0.9325
probability(No): 0.06

In [11]:
pd.DataFrame([results]).T.sort_values(by=0,ascending=False)

,0
hold,0.998499
ride,0.997199
carry,0.989013
throw,0.989013
pour,0.989013
pack,0.987568
write,0.985936
stir,0.966914
peel,0.966914
sip,0.962673
